# Quantiles

https://machinelearningmastery.com/curve-fitting-with-python/

https://study.com/academy/lesson/interpolation-in-statistics-definition-formula-example.html

In [42]:
## Load source files

In [43]:
import numpy as np
import pandas as pd
import random

In [44]:
!pwd

/Users/satwant/Documents/Shapelet_method/Shapelet_Methods/Classification_task/Codes


In [45]:
Source_path = "../Data_Sources/quantile_preprocessing/"

In [46]:
test_file = pd.read_csv(Source_path+"2020-08-09-CEID-Walk.csv")

In [51]:
runtype = "case"

In [55]:
## filter for runtype
cases_data_tmp = test_file[test_file['target'].str.contains(runtype)]

## filter for quantiles only 

cases_data = cases_data_tmp[cases_data_tmp['type']=='quantile']


In [66]:
cases_data

,forecast_date,target,target_end_date,location,type,quantile,value
48,2020-08-09,1 wk ahead inc case,2020-08-15,01,quantile,0.025,5962.505
49,2020-08-09,1 wk ahead inc case,2020-08-15,01,quantile,0.100,7406.708
50,2020-08-09,1 wk ahead inc case,2020-08-15,01,quantile,0.250,8877.014
51,2020-08-09,1 wk ahead inc case,2020-08-15,01,quantile,0.500,10429.088
52,2020-08-09,1 wk ahead inc case,2020-08-15,01,quantile,0.750,12129.189
...,...,...,...,...,...,...,...
113274,2020-08-09,4 wk ahead inc case,2020-09-05,US,quantile,0.250,299860.867
113275,2020-08-09,4 wk ahead inc case,2020-09-05,US,quantile,0.500,369177.715
113276,2020-08-09,4 wk ahead inc case,2020-09-05,US,quantile,0.750,429681.458
113277,2020-08-09,4 wk ahead inc case,2020-09-05,US,quantile,0.900,516613.934


In [68]:
cases_data['ordered_pair'] =cases_data[['quantile', 'value']].apply(tuple, axis=1)



<ipython-input-68-0835ed7e860b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cases_data['ordered_pair'] =cases_data[['quantile', 'value']].apply(tuple, axis=1)


In [78]:
cases_subset = cases_data[['forecast_date','location','target','ordered_pair','target_end_date']]


cases_rolledup = cases_subset.groupby(['forecast_date','location','target','target_end_date'])['ordered_pair'].apply(list).reset_index()



In [81]:
pd.set_option('display.max_columns', None)


In [86]:

test_array = [(0.025, 5962.505), (0.1, 7406.708), (0.25, 8877.014), (0.5, 10429.088), (0.75, 12129.189), (0.9, 13698.82), (0.975, 14713.544)]



In [227]:
def interpolate(test_array):
    array_x = [w[0] for w in test_array]
    array_y = [w[1] for w in test_array]
    sum_iter = 0
    for i in range(50):
        value_x = np.random.uniform(low=0.0, high=1.0)
        sum_iter+=np.interp(value_x,array_x,array_y)
    return sum_iter/50


In [218]:
cases_rolledup['IR_value_50sim'] = cases_rolledup['ordered_pair'].apply(interpolate)

In [225]:
# cases_subset[(cases_subset['location']=='01') & (cases_subset['target']=='4 wk ahead inc case')]

,forecast_date,location,target,ordered_pair,target_end_date
216,2020-08-09,01,4 wk ahead inc case,"(0.025, 0.638)",2020-09-05
217,2020-08-09,01,4 wk ahead inc case,"(0.1, 3186.287)",2020-09-05
218,2020-08-09,01,4 wk ahead inc case,"(0.25, 6737.06)",2020-09-05
219,2020-08-09,01,4 wk ahead inc case,"(0.5, 9674.841)",2020-09-05
220,2020-08-09,01,4 wk ahead inc case,"(0.75, 13341.656)",2020-09-05
221,2020-08-09,01,4 wk ahead inc case,"(0.9, 15835.248)",2020-09-05
222,2020-08-09,01,4 wk ahead inc case,"(0.975, 17482.474)",2020-09-05


In [226]:
cases_rolledup

,forecast_date,location,target,target_end_date,ordered_pair,IR_value_50sim
0,2020-08-09,01,1 wk ahead inc case,2020-08-15,"[(0.025, 5962.505), (0.1, 7406.708), (0.25, 88...",10063.472173
1,2020-08-09,01,2 wk ahead inc case,2020-08-22,"[(0.025, 3829.026), (0.1, 5465.961), (0.25, 77...",10067.909694
2,2020-08-09,01,3 wk ahead inc case,2020-08-29,"[(0.025, 2733.087), (0.1, 4876.558), (0.25, 74...",9931.869003
3,2020-08-09,01,4 wk ahead inc case,2020-09-05,"[(0.025, 0.638), (0.1, 3186.287), (0.25, 6737....",9827.519538
4,2020-08-09,01001,1 wk ahead inc case,2020-08-15,"[(0.025, 41.866), (0.1, 60.519), (0.25, 80.804...",96.476641
...,...,...,...,...,...,...
12787,2020-08-09,78,4 wk ahead inc case,2020-09-05,"[(0.025, 0.0), (0.1, 0.0), (0.25, 36.994), (0....",129.874003
12788,2020-08-09,US,1 wk ahead inc case,2020-08-15,"[(0.025, 290982.679), (0.1, 309896.596), (0.25...",370910.027943
12789,2020-08-09,US,2 wk ahead inc case,2020-08-22,"[(0.025, 261987.462), (0.1, 280654.355), (0.25...",385625.294584
12790,2020-08-09,US,3 wk ahead inc case,2020-08-29,"[(0.025, 200147.439), (0.1, 257524.742), (0.25...",399655.583194


In [263]:
processed_df = cases_rolledup.pivot(index=['forecast_date','location'], columns='target_end_date', values='IR_value_50sim').reset_index()



In [265]:
current_date = processed_df.forecast_date.unique()[0]

In [266]:
processed_df.replace(current_date,"NaN",inplace=True)

In [267]:
original_columns = list(processed_df.columns)

In [271]:
processed_df.columns = [current_date]+[original_columns[1]]+original_columns[2:]

In [273]:
processed_df_final = processed_df[[original_columns[1]]+[current_date]+original_columns[2:]]

In [274]:
processed_df_final

,location,2020-08-09,2020-08-15,2020-08-22,2020-08-29,2020-09-05
0,01,NaN,10063.472173,10067.909694,9931.869003,9827.519538
1,01001,NaN,96.476641,104.130352,107.947853,115.464909
2,01003,NaN,484.352074,416.586318,447.520027,468.479518
3,01005,NaN,26.453318,17.391378,15.118307,21.016704
4,01007,NaN,63.969214,65.559894,65.205716,63.618655
...,...,...,...,...,...,...
3193,66,NaN,57.058448,54.910670,50.342924,51.444087
3194,69,NaN,4.185623,3.814157,4.110112,4.133898
3195,72,NaN,3542.295718,3457.178047,3553.352069,3502.905559
3196,78,NaN,111.090125,123.443248,97.755159,129.874003
